In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install torch onnx onnx-tf tensorflow

INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (787 bytes)
  Using cached tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 49.4 MB/s eta 0:00:00
Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 23.7 MB/s eta 0:00:00
Using cached tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl (4.

In [3]:
%cd /content/drive/MyDrive/Colab_Notebooks/The model

/content/drive/MyDrive/Colab_Notebooks/The model


In [6]:
!python export.py --weights best.pt --img 640 --batch 1 --device cpu --include onnx

export: data=data/coco128.yaml, weights=['best.pt'], imgsz=[640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, simplify=False, mlmodel=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
fatal: cannot change to '/content/drive/MyDrive/Colab_Notebooks/The': No such file or directory
YOLOv5 🚀 2024-10-5 Python-3.10.12 torch-2.4.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from best.pt with output shape (1, 25200, 9) (13.8 MB)

ONNX: starting export with onnx 1.17.0...
ONNX: export success ✅ 1.5s, saved as best.onnx (27.2 MB)

Export complete (4.3s)
Results saved to /content/drive/MyDrive/Colab_Notebooks/The model
Detect:          python detect.py --weights best.onnx 
Validate:        python val.py --weights best.onnx 
PyTorch 

In [5]:
import onnx
from onnx_tf.backend import prepare
import tensorflow as tf

# Step 1: Load the ONNX model
onnx_model = onnx.load("best.onnx")

# Step 2: Convert ONNX model to TensorFlow
tf_rep = prepare(onnx_model)

# Step 3: Save the TensorFlow model
tf_rep.export_graph("tf_model")

In [6]:
# Step 4: Load the saved model
converter = tf.lite.TFLiteConverter.from_saved_model("tf_model")

# Step 5: Configure the converter to allow Flex ops
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TFLite ops
    tf.lite.OpsSet.SELECT_TF_OPS     # Enable TF ops
]

# Optional: If you need to reduce model size, you can try quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Step 6: Convert to TFLite
tflite_model = converter.convert()

# Step 7: Save the TFLite model
with open("converted_model.tflite", "wb") as f:
    f.write(tflite_model)